
# Kernel for CareerCon 2019

## Author: Min Yang 
## NetId: my2138

## Use the deep learning algorithms to solve the problem 

In this competition we find our data has time-series, so I decide to use LSTM as our model. Long Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies. They are explicitly designed to avoid the long-term dependency problem. Remembering information for long periods of time is practically their default behavior, not something they struggle to learn. All recurrent neural networks have the form of a chain of repeating modules of neural network. In standard RNNs, this repeating module will have a very simple structure, such as a single tanh layer. LSTMs also have this chain like structure, but the repeating module has a different structure. Instead of having a single neural network layer, there are four, interacting in a very special way. Like most RNNs, an LSTM network is universal in the sense that given enough network units it can compute anything aconventional computer can compute, provided it has the proper weight matrix, which may be viewed as its program. Unlike traditional RNNs, an LSTM network is well-suited to learn from experience to classify, process and predict time series when there are very long time lags of unknown size between important events. This is one of the main reasons why LSTM outperforms alternative RNNs.

### The Core Idea Behind LSTMS
The key to LSTMs is the cell state, the horizontal line running through the top of the diagram. The cell state is kind of like a conveyor belt. It runs straight down the entire chain, with only some minor linear interactions. It’s very easy for information to just flow along it unchanged. The LSTM does have the ability to remove or add information to the cell state, carefully regulated by structures called gates. Gates are a way to optionally let information through. They are composed out of a sigmoid neural net layer and a pointwise multiplication operation. The sigmoid layer outputs numbers between zero and one, describing how much of each component should be let through. A value of zero means “let nothing through,” while a value of one means “let everything through!”. An LSTM has three of these gates, to protect and control the cell state.


### The structure of the LSTM
* Forget Gate Layer

The first step in our LSTM is to decide what information we’re going to throw away from the cell state. This decision is made by a sigmoid layer called the “forget gate layer.” It looks at $h_(t-1)$ and $x_t$, and outputs a number between 0 and 1 for each number in the cell state $C_(t-1)$. A 1 represents “completely keep this” while a 0 represents “completely get rid of this.”

* Input Gate Layer

The next step is to decide what new information we’re going to store in the cell state. This has two parts. First, a sigmoid layer called the “input gate layer” decides which values we’ll update. Next, a tanh layer creates a vector of new candidate values, $C_t$ , that could be added to the state. In the next step, we’ll combine these two to create an update to the state.

* Update

It’s now time to update the old cell state, $C_(t-1)$, into the new cell state $C_t$. The previous steps already decided what to do, we just need to actually do it. We multiply the old state by ft, forgetting the things we decided to forget earlier. Then we add it∗$C_t$. This is the new candidate values, scaled by how much we decided to update each state value.

* Output

Finally, we need to decide what we’re going to output. This output will be based on our cell state, but will be a filtered version. First, we run a sigmoid layer which decides what parts of the cell state we’re going to output. Then, we put the cell state through tanh (to push the values to be between −1 and 1) and multiply it by the output of the sigmoid gate, so that we only output the parts we decided to.










In [1]:
#import packages 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
import os
import statistics
import copy

Use GPU can incredibly boost the speed of training

In [2]:
#use GPU
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

After reading all the data by pandas, I build the relationship between the series_id and surface type and store all of them in the dictionary. Then I change all the data to the tensor type for the later network to use.

In [3]:
#prepare the data
X_train_new = pd.read_csv("../input/robot-dataset-ai/X_train_new.csv")
y_train_new = pd.read_csv("../input/robot-dataset-ai/y_train_new.csv")
X_test_new = pd.read_csv("../input/robot-dataset-ai/X_test_new.csv")
y_test_new = pd.read_csv("../input/robot-dataset-ai/y_test_new.csv")


series_id_surface = {} # store the relation of series_id -> surface
surface_series_id = {} # store the relation of surface -> series_id

test_series_id_surface = {} # For the test dataset

for r in y_train_new.iterrows():
    series_id_surface[r[1].series_id] = r[1].surface
    surface_series_id.setdefault(r[1].surface,[]).append(r[1].series_id)
    
for r in y_test_new.iterrows():
    test_series_id_surface[r[1].series_id] = r[1].surface
    
print(test_series_id_surface)

surface_types = list(surface_series_id.keys())
surfaces_num = len(surface_types)
features_num = X_train_new.drop(['row_id', 'series_id', 'measurement_number'], axis=1).shape[1]
series_num = len(X_train_new['series_id'].unique())
measurement_num = len(X_train_new['measurement_number'].unique())
hidden_num = 128


{3790: 'hard_tiles_large_space', 3791: 'concrete', 3792: 'soft_pvc', 3793: 'tiled', 3794: 'hard_tiles_large_space', 3795: 'concrete', 3796: 'wood', 3797: 'fine_concrete', 3798: 'concrete', 3799: 'tiled', 3800: 'fine_concrete', 3801: 'hard_tiles_large_space', 3802: 'tiled', 3803: 'wood', 3804: 'tiled', 3805: 'tiled', 3806: 'wood', 3807: 'fine_concrete', 3808: 'tiled', 3809: 'soft_pvc'}


Then I use the function of "train_test_split" in the sklearn module to get the evaluation dataset and train dataset.

In [4]:
#split the dataset
datasets = {}
datasets["train"], datasets["eval"] = train_test_split(X_train_new['series_id'].unique(), test_size = 0.1) 

In [5]:
#change to tensor
surface_to_tensor = {}
for i in range(surfaces_num):
    surface_to_tensor[surface_types[i]] = torch.tensor([i], dtype=torch.long).to(device)

def get_tensor_by_series_id(series_id, dataframe):
    series = dataframe[dataframe['series_id'] == series_id].drop(['row_id', 'series_id', 'measurement_number'], axis=1)
    series_tensor = torch.from_numpy(series.values).view(series.shape[0], 1, -1).float().to(device)
    return series_tensor

def produce_training_example(phase):
    series_id = random.choice(datasets[phase])
    surface = series_id_surface[series_id]
    surface_tensor = surface_to_tensor[surface]
    series_tensor = series_id_to_tensor[series_id]
    return series_id, surface, series_tensor, surface_tensor

series_id_to_tensor = {}
for i in X_train_new['series_id'].unique():
    series_id_to_tensor[i] = get_tensor_by_series_id(i, X_train_new)


In [6]:
#get the labels set
labels = {}
labels['train'] = [series_id_surface[i] for i in datasets['train']]
labels['eval'] = [series_id_surface[i] for i in datasets['eval']]

In [7]:
class LSTM(nn.Module):
    def __init__(self, series_dim, hidden_dim, surface_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(series_dim, hidden_dim, num_layers=2, dropout=0.2)
        self.hidden2surface = nn.Linear(hidden_dim, surface_dim)

    def forward(self, series):
        lstm_out, _ = self.lstm(series)
        surface_space = self.hidden2surface(lstm_out)
        surface_scores = F.log_softmax(surface_space, dim=2)
        return surface_scores[-1]

In [8]:
def get_surface(output):
    top_n, top_i = output.topk(1)
    surface_i = top_i[0].item()
    return surface_types[surface_i], surface_i

model = LSTM(features_num, hidden_num, surfaces_num).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())

In [9]:
#train and val
epoches = 200000
save_loss_epoches = 10
print_epoches = 1000

losses = {'train': [], 'eval': []}
accs = {'train': [], 'eval': []}

all_losses = {'train': [], 'eval': []}

best_acc = 0.0

best_model_param = copy.deepcopy(model.state_dict()) # copy the param of the model and save
for epoch in range(1, epoches + 1):
    for phase in ['train', 'eval']:
        optimizer.zero_grad()
        if phase == 'train':
            model.train()
        else:
            model.eval()
        
        _,_,series,labels = produce_training_example(phase)
        
        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(series)
            loss = criterion(outputs, labels)
            _,preds = torch.max(outputs,1)
            if phase == 'train':
                loss.backward()
                optimizer.step()
                
        if epoch % save_loss_epoches == 0:
            loss = series.size(1) * loss.item() / len(labels)
            acc = torch.sum(preds == labels.data).double() / len(labels)
            losses[phase].append(loss)
            accs[phase].append(acc.item())
            
        if phase == 'eval' and epoch % print_epoches == 0:
            training_loss = statistics.mean(losses['train'])    #we add one period of time's loss and acc together and get the mean, then empty all of them
            validation_loss = statistics.mean(losses['eval'])
            all_losses['train'].append(training_loss)
            all_losses['eval'].append(validation_loss)
            training_acc = statistics.mean(accs['train'])
            validation_acc = statistics.mean(accs['eval'])
            if validation_acc > best_acc:
                best_acc = validation_acc
                print(f'Best accuracy so far! Acc = {best_acc}')
                best_model_param = copy.deepcopy(model.state_dict())
            print(f'{epoch} {epoch / epoches:.2%} / Train Loss: {training_loss:.4}, Validation Loss: {validation_loss:.4} / Train Accuracy: {training_acc:.4}, Validation Accuracy: {validation_acc:.4}')
            losses = {'train': [], 'eval': []}
            accs = {'train': [], 'eval': []}
            
            


Best accuracy so far! Acc = 0.16
1000 0.50% / Train Loss: 2.021, Validation Loss: 2.118 / Train Accuracy: 0.23, Validation Accuracy: 0.16
Best accuracy so far! Acc = 0.23
2000 1.00% / Train Loss: 2.079, Validation Loss: 2.039 / Train Accuracy: 0.2, Validation Accuracy: 0.23
Best accuracy so far! Acc = 0.26
3000 1.50% / Train Loss: 1.997, Validation Loss: 1.925 / Train Accuracy: 0.25, Validation Accuracy: 0.26
4000 2.00% / Train Loss: 1.797, Validation Loss: 1.95 / Train Accuracy: 0.3, Validation Accuracy: 0.24
Best accuracy so far! Acc = 0.38
5000 2.50% / Train Loss: 1.729, Validation Loss: 1.616 / Train Accuracy: 0.37, Validation Accuracy: 0.38
6000 3.00% / Train Loss: 1.552, Validation Loss: 1.831 / Train Accuracy: 0.37, Validation Accuracy: 0.29
7000 3.50% / Train Loss: 1.601, Validation Loss: 1.668 / Train Accuracy: 0.41, Validation Accuracy: 0.32
Best accuracy so far! Acc = 0.43
8000 4.00% / Train Loss: 1.534, Validation Loss: 1.492 / Train Accuracy: 0.36, Validation Accuracy: 0.4

In [10]:
#test part
def test(input):
    with torch.no_grad():
        return model(input)
    
preds = []
correct = 0
train_labels_num = 0
for i in X_test_new['series_id'].unique():
    output = test(get_tensor_by_series_id(i, X_test_new))
    pred, pred_i = get_surface(output)
    if pred == test_series_id_surface[i]:
        correct += 1
    preds.append((i, pred))
    train_labels_num += 1

test_acc = correct / train_labels_num
print(f'Test accuracy  Acc = {test_acc}')
submission = pd.DataFrame(preds, columns=['series_id', 'surface'])
print(submission.to_csv(index=False))
submission.to_csv('submission_final.csv', index=False)


Test accuracy  Acc = 0.8
series_id,surface
3790,hard_tiles_large_space
3791,concrete
3792,soft_pvc
3793,tiled
3794,hard_tiles_large_space
3795,fine_concrete
3796,wood
3797,concrete
3798,concrete
3799,tiled
3800,fine_concrete
3801,hard_tiles_large_space
3802,tiled
3803,wood
3804,tiled
3805,tiled
3806,wood
3807,tiled
3808,wood
3809,soft_pvc



Here is the training and evaluation log:  
Best accuracy so far! Acc = 0.29  
1000 0.50% / Train Loss: 2.042, Validation Loss: 1.899 / Train Accuracy: 0.22, Validation Accuracy: 0.29  
Best accuracy so far! Acc = 0.37  
2000 1.00% / Train Loss: 1.947, Validation Loss: 1.862 / Train Accuracy: 0.22, Validation Accuracy: 0.37  
3000 1.50% / Train Loss: 1.8, Validation Loss: 1.848 / Train Accuracy: 0.26, Validation Accuracy: 0.31  
4000 2.00% / Train Loss: 1.818, Validation Loss: 1.698 / Train Accuracy: 0.32, Validation Accuracy: 0.32  
5000 2.50% / Train Loss: 1.596, Validation Loss: 1.771 / Train Accuracy: 0.36, Validation Accuracy: 0.32  
6000 3.00% / Train Loss: 1.593, Validation Loss: 1.604 / Train Accuracy: 0.34, Validation Accuracy: 0.32  
7000 3.50% / Train Loss: 1.591, Validation Loss: 1.693 / Train Accuracy: 0.4, Validation Accuracy: 0.33  
Best accuracy so far! Acc = 0.4  
8000 4.00% / Train Loss: 1.435, Validation Loss: 1.429 / Train Accuracy: 0.44, Validation Accuracy: 0.4  
9000 4.50% / Train Loss: 1.526, Validation Loss: 1.488 / Train Accuracy: 0.36, Validation Accuracy: 0.4  
10000 5.00% / Train Loss: 1.572, Validation Loss: 1.387 / Train Accuracy: 0.45, Validation Accuracy: 0.38  
Best accuracy so far! Acc = 0.49  
11000 5.50% / Train Loss: 1.622, Validation Loss: 1.367 / Train Accuracy: 0.39, Validation Accuracy: 0.49  
12000 6.00% / Train Loss: 1.444, Validation Loss: 1.492 / Train Accuracy: 0.37, Validation Accuracy: 0.39  
13000 6.50% / Train Loss: 1.462, Validation Loss: 1.433 / Train Accuracy: 0.47, Validation Accuracy: 0.4  
14000 7.00% / Train Loss: 1.417, Validation Loss: 1.37 / Train Accuracy: 0.45, Validation Accuracy: 0.45  
15000 7.50% / Train Loss: 1.387, Validation Loss: 1.343 / Train Accuracy: 0.46, Validation Accuracy: 0.49  
16000 8.00% / Train Loss: 1.297, Validation Loss: 1.548 / Train Accuracy: 0.53, Validation Accuracy: 0.38  
Best accuracy so far! Acc = 0.54  
17000 8.50% / Train Loss: 1.257, Validation Loss: 1.305 / Train Accuracy: 0.55, Validation Accuracy: 0.54  
Best accuracy so far! Acc = 0.56  
18000 9.00% / Train Loss: 1.241, Validation Loss: 1.364 / Train Accuracy: 0.53, Validation Accuracy: 0.56  
Best accuracy so far! Acc = 0.58  
19000 9.50% / Train Loss: 1.276, Validation Loss: 1.239 / Train Accuracy: 0.54, Validation Accuracy: 0.58  
20000 10.00% / Train Loss: 1.381, Validation Loss: 1.229 / Train Accuracy: 0.49, Validation Accuracy: 0.51  
21000 10.50% / Train Loss: 1.276, Validation Loss: 1.445 / Train Accuracy: 0.5, Validation Accuracy: 0.46  
22000 11.00% / Train Loss: 1.172, Validation Loss: 1.326 / Train Accuracy: 0.56, Validation Accuracy: 0.54  
23000 11.50% / Train Loss: 1.224, Validation Loss: 1.388 / Train Accuracy: 0.52, Validation Accuracy: 0.58  
Best accuracy so far! Acc = 0.61  
24000 12.00% / Train Loss: 1.174, Validation Loss: 1.227 / Train Accuracy: 0.59, Validation Accuracy: 0.61  
25000 12.50% / Train Loss: 1.23, Validation Loss: 1.189 / Train Accuracy: 0.53, Validation Accuracy: 0.59  
26000 13.00% / Train Loss: 1.006, Validation Loss: 1.27 / Train Accuracy: 0.6, Validation Accuracy: 0.56  
27000 13.50% / Train Loss: 1.056, Validation Loss: 1.33 / Train Accuracy: 0.64, Validation Accuracy: 0.53  
28000 14.00% / Train Loss: 1.29, Validation Loss: 1.156 / Train Accuracy: 0.53, Validation Accuracy: 0.56  
29000 14.50% / Train Loss: 1.009, Validation Loss: 1.24 / Train Accuracy: 0.64, Validation Accuracy: 0.57  
30000 15.00% / Train Loss: 1.032, Validation Loss: 1.118 / Train Accuracy: 0.62, Validation Accuracy: 0.58  
31000 15.50% / Train Loss: 1.059, Validation Loss: 1.215 / Train Accuracy: 0.64, Validation Accuracy: 0.58  
32000 16.00% / Train Loss: 1.156, Validation Loss: 1.029 / Train Accuracy: 0.57, Validation Accuracy: 0.61  
33000 16.50% / Train Loss: 0.9572, Validation Loss: 1.298 / Train Accuracy: 0.62, Validation Accuracy: 0.52  
Best accuracy so far! Acc = 0.64  
34000 17.00% / Train Loss: 1.013, Validation Loss: 1.183 / Train Accuracy: 0.64, Validation Accuracy: 0.64  
35000 17.50% / Train Loss: 1.014, Validation Loss: 1.205 / Train Accuracy: 0.62, Validation Accuracy: 0.61  
36000 18.00% / Train Loss: 0.8353, Validation Loss: 1.175 / Train Accuracy: 0.69, Validation Accuracy: 0.62  
37000 18.50% / Train Loss: 0.7571, Validation Loss: 1.051 / Train Accuracy: 0.76, Validation Accuracy: 0.61  
38000 19.00% / Train Loss: 0.8978, Validation Loss: 1.207 / Train Accuracy: 0.72, Validation Accuracy: 0.57  
39000 19.50% / Train Loss: 0.9831, Validation Loss: 1.154 / Train Accuracy: 0.66, Validation Accuracy: 0.58  
40000 20.00% / Train Loss: 0.7601, Validation Loss: 1.044 / Train Accuracy: 0.72, Validation Accuracy: 0.6  
41000 20.50% / Train Loss: 0.7609, Validation Loss: 0.9779 / Train Accuracy: 0.7, Validation Accuracy: 0.59  
Best accuracy so far! Acc = 0.69  
42000 21.00% / Train Loss: 0.8485, Validation Loss: 0.9171 / Train Accuracy: 0.7, Validation Accuracy: 0.69  
43000 21.50% / Train Loss: 0.726, Validation Loss: 1.035 / Train Accuracy: 0.72, Validation Accuracy: 0.62  
44000 22.00% / Train Loss: 1.017, Validation Loss: 1.008 / Train Accuracy: 0.61, Validation Accuracy: 0.64  
45000 22.50% / Train Loss: 0.7086, Validation Loss: 0.8274 / Train Accuracy: 0.73, Validation Accuracy: 0.66  
Best accuracy so far! Acc = 0.71  
46000 23.00% / Train Loss: 0.6345, Validation Loss: 0.8965 / Train Accuracy: 0.76, Validation Accuracy: 0.71  
47000 23.50% / Train Loss: 0.7755, Validation Loss: 1.038 / Train Accuracy: 0.74, Validation Accuracy: 0.63  
48000 24.00% / Train Loss: 0.8372, Validation Loss: 1.005 / Train Accuracy: 0.62, Validation Accuracy: 0.66  
49000 24.50% / Train Loss: 0.7922, Validation Loss: 0.8637 / Train Accuracy: 0.69, Validation Accuracy: 0.65  
50000 25.00% / Train Loss: 0.6593, Validation Loss: 0.968 / Train Accuracy: 0.74, Validation Accuracy: 0.67  
51000 25.50% / Train Loss: 0.6949, Validation Loss: 0.8857 / Train Accuracy: 0.75, Validation Accuracy: 0.68  
52000 26.00% / Train Loss: 0.7691, Validation Loss: 1.12 / Train Accuracy: 0.73, Validation Accuracy: 0.59  
53000 26.50% / Train Loss: 0.6602, Validation Loss: 0.8045 / Train Accuracy: 0.8, Validation Accuracy: 0.71  
54000 27.00% / Train Loss: 0.6846, Validation Loss: 0.7781 / Train Accuracy: 0.75, Validation Accuracy: 0.71  
55000 27.50% / Train Loss: 0.8085, Validation Loss: 1.025 / Train Accuracy: 0.7, Validation Accuracy: 0.63  
56000 28.00% / Train Loss: 0.7359, Validation Loss: 0.7915 / Train Accuracy: 0.78, Validation Accuracy: 0.7    
57000 28.50% / Train Loss: 0.7046, Validation Loss: 1.062 / Train Accuracy: 0.71, Validation Accuracy: 0.69  
58000 29.00% / Train Loss: 0.6393, Validation Loss: 1.051 / Train Accuracy: 0.76, Validation Accuracy: 0.66  
59000 29.50% / Train Loss: 0.9194, Validation Loss: 0.9103 / Train Accuracy: 0.64, Validation Accuracy: 0.7  
60000 30.00% / Train Loss: 0.6196, Validation Loss: 0.9834 / Train Accuracy: 0.76, Validation Accuracy: 0.65  
61000 30.50% / Train Loss: 0.6884, Validation Loss: 1.047 / Train Accuracy: 0.76, Validation Accuracy: 0.62  
62000 31.00% / Train Loss: 0.4688, Validation Loss: 0.6798 / Train Accuracy: 0.85, Validation Accuracy: 0.71  
Best accuracy so far! Acc = 0.73  
63000 31.50% / Train Loss: 0.6026, Validation Loss: 0.7924 / Train Accuracy: 0.74, Validation Accuracy: 0.73  
64000 32.00% / Train Loss: 0.736, Validation Loss: 0.9608 / Train Accuracy: 0.75, Validation Accuracy: 0.64  
65000 32.50% / Train Loss: 0.7503, Validation Loss: 1.207 / Train Accuracy: 0.68, Validation Accuracy: 0.64  
66000 33.00% / Train Loss: 0.5858, Validation Loss: 0.8172 / Train Accuracy: 0.78, Validation Accuracy: 0.72  
67000 33.50% / Train Loss: 0.5349, Validation Loss: 1.087 / Train Accuracy: 0.77, Validation Accuracy: 0.61  
68000 34.00% / Train Loss: 0.5361, Validation Loss: 0.9116 / Train Accuracy: 0.8, Validation Accuracy: 0.71  
69000 34.50% / Train Loss: 0.6117, Validation Loss: 1.127 / Train Accuracy: 0.79, Validation Accuracy: 0.61  
70000 35.00% / Train Loss: 0.5716, Validation Loss: 0.8805 / Train Accuracy: 0.78, Validation Accuracy: 0.72  
71000 35.50% / Train Loss: 0.5411, Validation Loss: 1.073 / Train Accuracy: 0.81, Validation Accuracy: 0.68  
72000 36.00% / Train Loss: 0.4653, Validation Loss: 0.9709 / Train Accuracy: 0.82, Validation Accuracy: 0.65  
73000 36.50% / Train Loss: 0.5369, Validation Loss: 0.92 / Train Accuracy: 0.8, Validation Accuracy: 0.7  
Best accuracy so far! Acc = 0.8  
74000 37.00% / Train Loss: 0.433, Validation Loss: 0.7879 / Train Accuracy: 0.82, Validation Accuracy: 0.8  
75000 37.50% / Train Loss: 0.4265, Validation Loss: 0.7071 / Train Accuracy: 0.83, Validation Accuracy: 0.73  
76000 38.00% / Train Loss: 0.5545, Validation Loss: 1.16 / Train Accuracy: 0.8, Validation Accuracy: 0.65  
77000 38.50% / Train Loss: 0.5195, Validation Loss: 0.7537 / Train Accuracy: 0.81, Validation Accuracy: 0.78  
78000 39.00% / Train Loss: 0.4028, Validation Loss: 0.7867 / Train Accuracy: 0.85, Validation Accuracy: 0.72  
79000 39.50% / Train Loss: 0.4536, Validation Loss: 0.8891 / Train Accuracy: 0.82, Validation Accuracy: 0.77  
80000 40.00% / Train Loss: 0.5034, Validation Loss: 1.123 / Train Accuracy: 0.8, Validation Accuracy: 0.68  
81000 40.50% / Train Loss: 0.623, Validation Loss: 1.295 / Train Accuracy: 0.77, Validation Accuracy: 0.65  
82000 41.00% / Train Loss: 0.4301, Validation Loss: 1.007 / Train Accuracy: 0.87, Validation Accuracy: 0.67  
83000 41.50% / Train Loss: 0.6498, Validation Loss: 1.098 / Train Accuracy: 0.8, Validation Accuracy: 0.72  
84000 42.00% / Train Loss: 0.4358, Validation Loss: 1.077 / Train Accuracy: 0.84, Validation Accuracy: 0.62  
85000 42.50% / Train Loss: 0.4424, Validation Loss: 1.15 / Train Accuracy: 0.81, Validation Accuracy: 0.63  
86000 43.00% / Train Loss: 0.3176, Validation Loss: 0.8659 / Train Accuracy: 0.91, Validation Accuracy: 0.78  
87000 43.50% / Train Loss: 0.3644, Validation Loss: 0.9375 / Train Accuracy: 0.87, Validation Accuracy: 0.72  
88000 44.00% / Train Loss: 0.2843, Validation Loss: 0.9175 / Train Accuracy: 0.91, Validation Accuracy: 0.71  
89000 44.50% / Train Loss: 0.4208, Validation Loss: 1.134 / Train Accuracy: 0.82, Validation Accuracy: 0.71  
90000 45.00% / Train Loss: 0.3372, Validation Loss: 0.6728 / Train Accuracy: 0.89, Validation Accuracy: 0.77  
91000 45.50% / Train Loss: 0.5149, Validation Loss: 0.782 / Train Accuracy: 0.84, Validation Accuracy: 0.76  
92000 46.00% / Train Loss: 0.3762, Validation Loss: 0.8825 / Train Accuracy: 0.88, Validation Accuracy: 0.74  
93000 46.50% / Train Loss: 0.417, Validation Loss: 0.9684 / Train Accuracy: 0.86, Validation Accuracy: 0.74  
94000 47.00% / Train Loss: 0.5191, Validation Loss: 1.161 / Train Accuracy: 0.8, Validation Accuracy: 0.65  
95000 47.50% / Train Loss: 0.321, Validation Loss: 1.347 / Train Accuracy: 0.88, Validation Accuracy: 0.69  
96000 48.00% / Train Loss: 0.4303, Validation Loss: 0.7935 / Train Accuracy: 0.85, Validation Accuracy: 0.75  
97000 48.50% / Train Loss: 0.2429, Validation Loss: 0.8841 / Train Accuracy: 0.91, Validation Accuracy: 0.71  
98000 49.00% / Train Loss: 0.3466, Validation Loss: 0.692 / Train Accuracy: 0.9, Validation Accuracy: 0.74  
99000 49.50% / Train Loss: 0.3112, Validation Loss: 0.7757 / Train Accuracy: 0.9, Validation Accuracy: 0.75  
100000 50.00% / Train Loss: 0.5249, Validation Loss: 0.8399 / Train Accuracy: 0.83, Validation Accuracy: 0.74  
101000 50.50% / Train Loss: 0.4714, Validation Loss: 0.8733 / Train Accuracy: 0.83, Validation Accuracy: 0.75  
102000 51.00% / Train Loss: 0.3011, Validation Loss: 0.6153 / Train Accuracy: 0.89, Validation Accuracy: 0.78  
103000 51.50% / Train Loss: 0.321, Validation Loss: 0.9484 / Train Accuracy: 0.91, Validation Accuracy: 0.64  
104000 52.00% / Train Loss: 0.3203, Validation Loss: 1.016 / Train Accuracy: 0.91, Validation Accuracy: 0.74  
105000 52.50% / Train Loss: 0.1743, Validation Loss: 0.801 / Train Accuracy: 0.92, Validation Accuracy: 0.75  
106000 53.00% / Train Loss: 0.332, Validation Loss: 0.8503 / Train Accuracy: 0.88, Validation Accuracy: 0.75  
107000 53.50% / Train Loss: 0.3471, Validation Loss: 0.9017 / Train Accuracy: 0.87, Validation Accuracy: 0.77  
108000 54.00% / Train Loss: 0.3279, Validation Loss: 1.173 / Train Accuracy: 0.88, Validation Accuracy: 0.66  
109000 54.50% / Train Loss: 0.2138, Validation Loss: 0.8594 / Train Accuracy: 0.9, Validation Accuracy: 0.76  
110000 55.00% / Train Loss: 0.2745, Validation Loss: 0.9582 / Train Accuracy: 0.92, Validation Accuracy: 0.71  
111000 55.50% / Train Loss: 0.3028, Validation Loss: 1.344 / Train Accuracy: 0.84, Validation Accuracy: 0.66  
112000 56.00% / Train Loss: 0.2439, Validation Loss: 0.8815 / Train Accuracy: 0.93, Validation Accuracy: 0.73  
113000 56.50% / Train Loss: 0.2314, Validation Loss: 0.7855 / Train Accuracy: 0.93, Validation Accuracy: 0.74  
114000 57.00% / Train Loss: 0.1396, Validation Loss: 0.7155 / Train Accuracy: 0.96, Validation Accuracy: 0.76  
115000 57.50% / Train Loss: 0.2117, Validation Loss: 1.078 / Train Accuracy: 0.93, Validation Accuracy: 0.68  
116000 58.00% / Train Loss: 0.4191, Validation Loss: 0.8203 / Train Accuracy: 0.87, Validation Accuracy: 0.75  
117000 58.50% / Train Loss: 0.3585, Validation Loss: 1.041 / Train Accuracy: 0.87, Validation Accuracy: 0.75  
118000 59.00% / Train Loss: 0.2045, Validation Loss: 1.195 / Train Accuracy: 0.94, Validation Accuracy: 0.72  
119000 59.50% / Train Loss: 0.2267, Validation Loss: 0.889 / Train Accuracy: 0.91, Validation Accuracy: 0.71  
120000 60.00% / Train Loss: 0.2994, Validation Loss: 1.112 / Train Accuracy: 0.92, Validation Accuracy: 0.71  
121000 60.50% / Train Loss: 0.4621, Validation Loss: 0.8102 / Train Accuracy: 0.87, Validation Accuracy: 0.76  
122000 61.00% / Train Loss: 0.2639, Validation Loss: 0.9113 / Train Accuracy: 0.94, Validation Accuracy: 0.75  
123000 61.50% / Train Loss: 0.3955, Validation Loss: 1.293 / Train Accuracy: 0.89, Validation Accuracy: 0.59  
124000 62.00% / Train Loss: 0.1239, Validation Loss: 0.9179 / Train Accuracy: 0.95, Validation Accuracy: 0.76  
Best accuracy so far! Acc = 0.81  
125000 62.50% / Train Loss: 0.2791, Validation Loss: 0.744 / Train Accuracy: 0.91, Validation Accuracy: 0.81  
126000 63.00% / Train Loss: 0.2153, Validation Loss: 1.07 / Train Accuracy: 0.93, Validation Accuracy: 0.7  
127000 63.50% / Train Loss: 0.1964, Validation Loss: 1.29 / Train Accuracy: 0.92, Validation Accuracy: 0.69  
128000 64.00% / Train Loss: 0.2467, Validation Loss: 0.976 / Train Accuracy: 0.89, Validation Accuracy: 0.71  
129000 64.50% / Train Loss: 0.1876, Validation Loss: 1.372 / Train Accuracy: 0.95, Validation Accuracy: 0.71  
130000 65.00% / Train Loss: 0.1977, Validation Loss: 0.918 / Train Accuracy: 0.89, Validation Accuracy: 0.8  
131000 65.50% / Train Loss: 0.1598, Validation Loss: 0.926 / Train Accuracy: 0.95, Validation Accuracy: 0.77  
132000 66.00% / Train Loss: 0.2165, Validation Loss: 0.9564 / Train Accuracy: 0.95, Validation Accuracy: 0.75  
133000 66.50% / Train Loss: 0.1944, Validation Loss: 1.409 / Train Accuracy: 0.91, Validation Accuracy: 0.66  
134000 67.00% / Train Loss: 0.1945, Validation Loss: 1.452 / Train Accuracy: 0.93, Validation Accuracy: 0.6  
135000 67.50% / Train Loss: 0.2931, Validation Loss: 1.028 / Train Accuracy: 0.94, Validation Accuracy: 0.73  
136000 68.00% / Train Loss: 0.2038, Validation Loss: 0.9324 / Train Accuracy: 0.94, Validation Accuracy: 0.76  
137000 68.50% / Train Loss: 0.2589, Validation Loss: 1.008 / Train Accuracy: 0.92, Validation Accuracy: 0.74  
138000 69.00% / Train Loss: 0.2101, Validation Loss: 1.072 / Train Accuracy: 0.92, Validation Accuracy: 0.73  
139000 69.50% / Train Loss: 0.2301, Validation Loss: 1.172 / Train Accuracy: 0.9, Validation Accuracy: 0.68  
140000 70.00% / Train Loss: 0.3843, Validation Loss: 0.6111 / Train Accuracy: 0.86, Validation Accuracy: 0.8  
141000 70.50% / Train Loss: 0.1578, Validation Loss: 1.199 / Train Accuracy: 0.96, Validation Accuracy: 0.71  
142000 71.00% / Train Loss: 0.1741, Validation Loss: 0.9048 / Train Accuracy: 0.95, Validation Accuracy: 0.75  
143000 71.50% / Train Loss: 0.3405, Validation Loss: 1.105 / Train Accuracy: 0.87, Validation Accuracy: 0.72  
144000 72.00% / Train Loss: 0.1648, Validation Loss: 0.9328 / Train Accuracy: 0.94, Validation Accuracy: 0.74  
145000 72.50% / Train Loss: 0.201, Validation Loss: 1.228 / Train Accuracy: 0.92, Validation Accuracy: 0.73  
146000 73.00% / Train Loss: 0.316, Validation Loss: 1.197 / Train Accuracy: 0.94, Validation Accuracy: 0.74  
147000 73.50% / Train Loss: 0.2466, Validation Loss: 0.985 / Train Accuracy: 0.91, Validation Accuracy: 0.71  
148000 74.00% / Train Loss: 0.2034, Validation Loss: 1.077 / Train Accuracy: 0.94, Validation Accuracy: 0.65  
149000 74.50% / Train Loss: 0.3076, Validation Loss: 0.9745 / Train Accuracy: 0.91, Validation Accuracy: 0.73  
150000 75.00% / Train Loss: 0.3711, Validation Loss: 1.197 / Train Accuracy: 0.88, Validation Accuracy: 0.74  
151000 75.50% / Train Loss: 0.3572, Validation Loss: 0.9296 / Train Accuracy: 0.9, Validation Accuracy: 0.72  
152000 76.00% / Train Loss: 0.2364, Validation Loss: 1.301 / Train Accuracy: 0.93, Validation Accuracy: 0.69  
153000 76.50% / Train Loss: 0.1288, Validation Loss: 0.9844 / Train Accuracy: 0.96, Validation Accuracy: 0.73  
154000 77.00% / Train Loss: 0.2011, Validation Loss: 1.303 / Train Accuracy: 0.94, Validation Accuracy: 0.69  
155000 77.50% / Train Loss: 0.1824, Validation Loss: 0.9501 / Train Accuracy: 0.92, Validation Accuracy: 0.76  
156000 78.00% / Train Loss: 0.3187, Validation Loss: 0.9408 / Train Accuracy: 0.88, Validation Accuracy: 0.78  
157000 78.50% / Train Loss: 0.1371, Validation Loss: 1.227 / Train Accuracy: 0.97, Validation Accuracy: 0.74  
158000 79.00% / Train Loss: 0.2339, Validation Loss: 1.152 / Train Accuracy: 0.93, Validation Accuracy: 0.78  
159000 79.50% / Train Loss: 0.1762, Validation Loss: 1.454 / Train Accuracy: 0.94, Validation Accuracy: 0.7  
160000 80.00% / Train Loss: 0.366, Validation Loss: 1.236 / Train Accuracy: 0.87, Validation Accuracy: 0.71  
161000 80.50% / Train Loss: 0.2256, Validation Loss: 1.124 / Train Accuracy: 0.9, Validation Accuracy: 0.77  
162000 81.00% / Train Loss: 0.27, Validation Loss: 1.249 / Train Accuracy: 0.9, Validation Accuracy: 0.71  
163000 81.50% / Train Loss: 0.213, Validation Loss: 1.152 / Train Accuracy: 0.89, Validation Accuracy: 0.73  
164000 82.00% / Train Loss: 0.1512, Validation Loss: 1.1 / Train Accuracy: 0.95, Validation Accuracy: 0.72  
165000 82.50% / Train Loss: 0.3272, Validation Loss: 1.513 / Train Accuracy: 0.87, Validation Accuracy: 0.69  
166000 83.00% / Train Loss: 0.2762, Validation Loss: 1.163 / Train Accuracy: 0.91, Validation Accuracy: 0.71  
167000 83.50% / Train Loss: 0.2504, Validation Loss: 0.8323 / Train Accuracy: 0.92, Validation Accuracy: 0.73  
168000 84.00% / Train Loss: 0.2557, Validation Loss: 0.7952 / Train Accuracy: 0.93, Validation Accuracy: 0.77  
169000 84.50% / Train Loss: 0.1059, Validation Loss: 1.113 / Train Accuracy: 0.95, Validation Accuracy: 0.72  
170000 85.00% / Train Loss: 0.1417, Validation Loss: 1.136 / Train Accuracy: 0.94, Validation Accuracy: 0.77  
171000 85.50% / Train Loss: 0.1508, Validation Loss: 1.227 / Train Accuracy: 0.94, Validation Accuracy: 0.68  
172000 86.00% / Train Loss: 0.2227, Validation Loss: 1.474 / Train Accuracy: 0.92, Validation Accuracy: 0.67  
173000 86.50% / Train Loss: 0.1948, Validation Loss: 0.9813 / Train Accuracy: 0.92, Validation Accuracy: 0.73  
174000 87.00% / Train Loss: 0.2974, Validation Loss: 1.543 / Train Accuracy: 0.94, Validation Accuracy: 0.71  
175000 87.50% / Train Loss: 0.2326, Validation Loss: 1.588 / Train Accuracy: 0.92, Validation Accuracy: 0.74  
176000 88.00% / Train Loss: 0.1972, Validation Loss: 1.374 / Train Accuracy: 0.95, Validation Accuracy: 0.71  
Best accuracy so far! Acc = 0.84  
177000 88.50% / Train Loss: 0.0909, Validation Loss: 0.8288 / Train Accuracy: 0.98, Validation Accuracy: 0.84  
178000 89.00% / Train Loss: 0.1176, Validation Loss: 0.821 / Train Accuracy: 0.95, Validation Accuracy: 0.79  
179000 89.50% / Train Loss: 0.2288, Validation Loss: 0.9858 / Train Accuracy: 0.91, Validation Accuracy: 0.79  
180000 90.00% / Train Loss: 0.135, Validation Loss: 1.006 / Train Accuracy: 0.94, Validation Accuracy: 0.77  
181000 90.50% / Train Loss: 0.04935, Validation Loss: 0.9617 / Train Accuracy: 0.99, Validation Accuracy: 0.8  
182000 91.00% / Train Loss: 0.2098, Validation Loss: 1.33 / Train Accuracy: 0.96, Validation Accuracy: 0.72  
183000 91.50% / Train Loss: 0.1858, Validation Loss: 1.138 / Train Accuracy: 0.94, Validation Accuracy: 0.8  
184000 92.00% / Train Loss: 0.1554, Validation Loss: 1.344 / Train Accuracy: 0.95, Validation Accuracy: 0.75  
185000 92.50% / Train Loss: 0.1077, Validation Loss: 1.341 / Train Accuracy: 0.98, Validation Accuracy: 0.73  
186000 93.00% / Train Loss: 0.1513, Validation Loss: 1.214 / Train Accuracy: 0.95, Validation Accuracy: 0.76  
187000 93.50% / Train Loss: 0.2138, Validation Loss: 1.287 / Train Accuracy: 0.94, Validation Accuracy: 0.71  
188000 94.00% / Train Loss: 0.1944, Validation Loss: 1.605 / Train Accuracy: 0.93, Validation Accuracy: 0.7  
189000 94.50% / Train Loss: 0.4407, Validation Loss: 1.012 / Train Accuracy: 0.88, Validation Accuracy: 0.74  
190000 95.00% / Train Loss: 0.1199, Validation Loss: 1.684 / Train Accuracy: 0.97, Validation Accuracy: 0.71  
191000 95.50% / Train Loss: 0.1576, Validation Loss: 1.276 / Train Accuracy: 0.96, Validation Accuracy: 0.71  
192000 96.00% / Train Loss: 0.139, Validation Loss: 1.168 / Train Accuracy: 0.95, Validation Accuracy: 0.7  
193000 96.50% / Train Loss: 0.1136, Validation Loss: 0.9763 / Train Accuracy: 0.96, Validation Accuracy: 0.7  
194000 97.00% / Train Loss: 0.1741, Validation Loss: 1.224 / Train Accuracy: 0.94, Validation Accuracy: 0.75  
195000 97.50% / Train Loss: 0.2104, Validation Loss: 1.14 / Train Accuracy: 0.95, Validation Accuracy: 0.76  
196000 98.00% / Train Loss: 0.2012, Validation Loss: 1.042 / Train Accuracy: 0.95, Validation Accuracy: 0.75  
197000 98.50% / Train Loss: 0.04406, Validation Loss: 1.683 / Train Accuracy: 0.98, Validation Accuracy: 0.66  
198000 99.00% / Train Loss: 0.1206, Validation Loss: 1.112 / Train Accuracy: 0.95, Validation Accuracy: 0.76  
199000 99.50% / Train Loss: 0.09875, Validation Loss: 1.187 / Train Accuracy: 0.97, Validation Accuracy: 0.71  
200000 100.00% / Train Loss: 0.2491, Validation Loss: 1.277 / Train Accuracy: 0.92, Validation Accuracy: 0.71  

The best validation accuracy is 0.84

Here is the test result of test data  
Test accuracy  Acc = 0.7  
series_id,surface  
3790,hard_tiles_large_space  
3791,concrete  
3792,soft_pvc  
3793,concrete  
3794,hard_tiles_large_space  
3795,fine_concrete  
3796,wood  
3797,soft_pvc  
3798,concrete  
3799,tiled  
3800,concrete  
3801,hard_tiles_large_space  
3802,tiled  
3803,wood  
3804,tiled  
3805,tiled  
3806,wood  
3807,wood  
3808,wood  
3809,soft_pvc  

My model can get the 0.84 in the validation process but only get the 0.7 accuracy in the test data, I think one biggest reason is the size of the test dataset. We can find that our test dataset only contains 20 series, it's a small number of data so I think the accuracy of testing process may exists some randomness.